In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets,layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset_url = '/content/drive/My Drive/dataset'


In [4]:
train_path = '/content/drive/My Drive/dataset/train'
test_path='/content/drive/My Drive/dataset/test'

In [ ]:
batch_size = 64
target_size = (48,48)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
val_datagen   = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

test_generator = val_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

In [6]:
import os

In [7]:
emotion_labels = sorted(os.listdir(train_path))
print(emotion_labels)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [8]:
input_shape = (48,48,1) # img_rows, img_colums, color_channels
num_classes = 7

In [9]:
from tensorflow.keras.optimizers import RMSprop,Adam

model = models.Sequential()
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (48,48,1)))
model.add(MaxPool2D(pool_size=(2,2)))

#
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

#
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

#
model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


#
model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#model.add(Dropout(0.2))

# 
model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation = "softmax"))
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])



# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape= (48,48,1))) #, data_format='channels_last', kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

# model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
# model.add(layers.Flatten())


# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

In [10]:
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [11]:
weight_path=os.path.join(dataset_url,"myweight")

try:
    os.mkdir(weight_path)
except:
    print('file exist')

file exist


In [12]:
num_epochs = 13
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_test   = test_generator.n//test_generator.batch_size
print(STEP_SIZE_TRAIN)

756


In [13]:
checkpoint_path = "/content/drive/My Drive/dataset/myweight"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.load_weights(checkpoint_dir)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)

In [14]:
model_train = model.fit(train_generator, epochs=num_epochs, verbose=1, 
                        validation_data=test_generator, callbacks=[cp_callback]
                        )

Epoch 1/13
757/757 [==============================] - ETA: 0s - loss: 0.7960 - accuracy: 0.7087 
Epoch 1: saving model to /content/drive/My Drive/dataset
757/757 [==============================] - 10742s 14s/step - loss: 0.7960 - accuracy: 0.7087 - val_loss: 1.1969 - val_accuracy: 0.5770
Epoch 2/13
757/757 [==============================] - ETA: 0s - loss: 0.7843 - accuracy: 0.7129
Epoch 2: saving model to /content/drive/My Drive/dataset
757/757 [==============================] - 648s 856ms/step - loss: 0.7843 - accuracy: 0.7129 - val_loss: 1.2116 - val_accuracy: 0.5807
Epoch 3/13
757/757 [==============================] - ETA: 0s - loss: 0.7758 - accuracy: 0.7165
Epoch 3: saving model to /content/drive/My Drive/dataset
757/757 [==============================] - 661s 873ms/step - loss: 0.7758 - accuracy: 0.7165 - val_loss: 1.2212 - val_accuracy: 0.5738
Epoch 4/13
757/757 [==============================] - ETA: 0s - loss: 0.7607 - accuracy: 0.7228
Epoch 4: saving model to /content/drive

In [19]:
model.save('/content/drive/My Drive/dataset/myweight/emotion_detection.h5')
model.save_weights('/content/drive/My Drive/dataset/myweight/model_weight.h5')

In [21]:
new_model = tf.keras.models.load_model('/content/drive/My Drive/dataset/myweight/emotion_detection.h5') #same file path